In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
import statistics
import time

In [2]:
img_dir = "sample_images" # Enter Directory of all images 
data_path = os.path.join(img_dir,'*')
img_files = glob.glob(data_path)
img_files

['sample_images/einstein.jpg',
 'sample_images/salt_pepper.jpg',
 'sample_images/Baboon.bmp',
 'sample_images/Lena.bmp',
 'sample_images/Cameraman.bmp']

In [3]:
#read & pad images and put them in seperate lists
img_list=[]
padded_list3=[]
padded_list5=[]
padded_list7=[]
padded_list11=[]
for s in img_files:
    img = cv2.imread(s,cv2.IMREAD_GRAYSCALE)
    #pad_width=(kernel-1)/2
    img3 = np.pad(img, pad_width=1, mode='constant', constant_values=0)
    img5 = np.pad(img, pad_width=2, mode='constant', constant_values=0)
    img7 = np.pad(img, pad_width=3, mode='constant', constant_values=0)
    img11 = np.pad(img, pad_width=5, mode='constant', constant_values=0)
    
    img_list.append(img)
    padded_list3.append(img3)
    padded_list5.append(img5)
    padded_list7.append(img7)
    padded_list11.append(img11)
    

In [4]:
#creating kernels
#Mean Kernels
kernel3x3=np.ones((3,3),np.float32)/9
kernel5x5=np.ones((5,5),np.float32)/25
kernel7x7=np.ones((7,7),np.float32)/49
kernel11x11=np.ones((11,11),np.float32)/121
#Kernel for shift 2 pixels up
shiftUp2Ker=np.zeros((5,5),np.float32)
shiftUp2Ker[4][2]=1
#Kernel for shift 3 pixels right
shiftRight3Ker=np.zeros((7,7),np.float32)
shiftRight3Ker[3][0]=1
#Kernel for sharpening
double_identical=np.zeros((3,3),np.float32)
double_identical[1][1]=2
sharp_kernel=double_identical-kernel3x3
#list for median filter
klist=[]

In [5]:
#create new images 
new_imgs=[]
npadded_list3=[]
npadded_list5=[]
npadded_list7=[]
npadded_list11=[]
for i in range(len(img_list)):
    row=img_list[i].shape[0]
    col=img_list[i].shape[1]
    new_imgs.append(np.zeros((row,col),np.uint8))
    
#Add pad to newly created images and put them in a list
for s in range(len(img_list)):
    nimg=new_imgs[s]
    
    nimg3 = np.pad(nimg, pad_width=1, mode='constant', constant_values=0)
    nimg5 = np.pad(nimg, pad_width=2, mode='constant', constant_values=0)
    nimg7 = np.pad(nimg, pad_width=3, mode='constant', constant_values=0)
    nimg11 = np.pad(nimg, pad_width=5, mode='constant', constant_values=0)
    

    npadded_list3.append(nimg3)
    npadded_list5.append(nimg5)
    npadded_list7.append(nimg7)
    npadded_list11.append(nimg11)

In [6]:


def OperationFunc(kernel_size,img_num,flagShiftUp,flagShiftRight,flagSharp,flagMedian):
    imagerow=new_imgs[img_num].shape[0] 
    imagecol=new_imgs[img_num].shape[1]
    #Apply filters by iterating through images
    for s in range(imagerow):
        for d in range(imagecol):
            sum=0
            for i in range(kernel_size):
                for j in range(kernel_size):
                    if flagShiftUp==True: #Apply shift up filter
                        ker_indx=shiftUp2Ker[i][j] #find kernel index  
                        nimg=padded_list5[img_num] #find image to apply
                        img_indx=nimg[s+i][d+j] #find image index
                        mult=ker_indx*img_indx #multiply kernel index and image index
                        
                        sum=sum+mult #the value that we're going to write in the middle
                        img=npadded_list5[img_num] #
                        img[s+int((kernel_size-1)/2)][d+int((kernel_size-1)/2)]=sum
                        npadded_list5[img_num]=img #write the calculated value to the right place in the image
#the logic after this part is same   
                    elif flagShiftRight==True: #Apply shift right filter
                        ker_indx=shiftRight3Ker[i][j]
                        nimg=padded_list7[img_num]
                        img_indx=nimg[s+i][d+j]
                        mult=ker_indx*img_indx
                        
                        sum=sum+mult
                        img=npadded_list7[img_num]
                        img[s+int((kernel_size-1)/2)][d+int((kernel_size-1)/2)]=sum
                        npadded_list7[img_num]=img
                        
                    elif flagSharp==True: #apply sharpening filter
                        ker_indx=sharp_kernel[i][j]
                        nimg=padded_list3[img_num]
                        img_indx=nimg[s+i][d+j]
                        mult=ker_indx*img_indx
                        
                        sum=sum+mult
                        img=npadded_list3[img_num]
                        img[s+int((kernel_size-1)/2)][d+int((kernel_size-1)/2)]=sum
                        npadded_list3[img_num]=img
                        
                    elif flagMedian==True: #apply median filter
                        nimg=padded_list3[img_num]
                        klist.append(nimg[s+i][d+j]) 
                    else: #appy mean filter
                        if kernel_size==3:
                            ker_indx=kernel3x3[i][j]
                            nimg=padded_list3[img_num]
                            img_indx=nimg[s+i][d+j]
                            mult=ker_indx*img_indx
                            #
                            sum=sum+mult
                            img=npadded_list3[img_num]
                            img[s+int((kernel_size-1)/2)][d+int((kernel_size-1)/2)]=sum
                            npadded_list3[img_num]=img
                            
                        elif kernel_size==5:
                            ker_indx=kernel5x5[i][j]
                            nimg=padded_list5[img_num]
                            img_indx=nimg[s+i][d+j]
                            mult=ker_indx*img_indx
                            
                            sum=sum+mult
                            img=npadded_list5[img_num]
                            img[s+int((kernel_size-1)/2)][d+int((kernel_size-1)/2)]=sum
                            npadded_list5[img_num]=img
                            
                        elif kernel_size==7:
                            ker_indx=kernel7x7[i][j]
                            nimg=padded_list7[img_num]
                            img_indx=nimg[s+i][d+j]
                            mult=ker_indx*img_indx
                            
                            sum=sum+mult
                            img=npadded_list7[img_num]
                            img[s+int((kernel_size-1)/2)][d+int((kernel_size-1)/2)]=sum
                            npadded_list7[img_num]=img
                            
                        elif kernel_size==11:
                            ker_indx=kernel11x11[i][j]
                            nimg=padded_list11[img_num]
                            img_indx=nimg[s+i][d+j]
                            mult=ker_indx*img_indx
                            
                            sum=sum+mult
                            img=npadded_list11[img_num]
                            img[s+int((kernel_size-1)/2)][d+int((kernel_size-1)/2)]=sum
                            npadded_list11[img_num]=img
                if flagMedian==True: #median filter continues
                    a=statistics.median(klist)#find median 
                    klist.clear()
                    img=npadded_list3[img_num]
                    img[s+int((kernel_size-1)/2)][d+int((kernel_size-1)/2)]=a
                    npadded_list3[img_num]=img
            

In [ ]:
#1 apply mean filter to images
#filter3x3
blur3=[]
for i in range(len(img_list)):
   
    OperationFunc(3,i,False,False,False,False)#
    blur3.append(npadded_list3[i])
    cv2.imshow("3x3Blur Original-Blur",np.hstack((padded_list3[i],blur3[i])))
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [11]:
#filter5x5
blur5=[]
for i in range(len(img_list)):
    
    filt5=OperationFunc(5,i,False,False,False,False)
    blur5.append(npadded_list5[i])
    cv2.imshow("5x5Blur Original-Blur",np.hstack((padded_list5[i],blur5[i])))
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    

In [13]:
#filter7x7
blur7=[]
for i in range(len(img_list)):

    filt7=OperationFunc(7,i,False,False,False,False)
    blur7.append(npadded_list7[i])
    cv2.imshow("7x7Blur Original-Blur",np.hstack((padded_list7[i],blur7[i])))
    cv2.waitKey(0)
    cv2.destroyAllWindows()
   

In [16]:
#filter11x11
blur11=[]
for i in range(len(img_list)):

    filt11=OperationFunc(11,i,False,False,False,False)
    blur11.append(npadded_list11[i])
    cv2.imshow("11x11Blur Original-Blur",np.hstack((padded_list11[i],blur11[i])))
    cv2.waitKey()
    cv2.destroyAllWindows()

In [10]:
#2 Subtract the blurred images obtained in question (1) from the original images
#3x3
sub3=[]
for i in range(len(img_list)):
    sub3.append(padded_list3[i]-blur3[i])
    cv2.imshow("3x3Sub Original-Subtracted",np.hstack((padded_list3[i],sub3[i])))
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [12]:
#5x5
sub5=[]
for i in range(len(img_list)):
    sub5.append(padded_list5[i]-blur5[i])
    cv2.imshow("5x5Sub Original-Subtracted",np.hstack((padded_list5[i],sub5[i])))
    cv2.waitKey(0)
    cv2.destroyAllWindows()


In [15]:
#7x7
sub7=[]
for i in range(len(img_list)):
    sub7.append(padded_list7[i]-blur7[i])
    cv2.imshow("7x7Sub Original-Subtracted",np.hstack((padded_list7[i],sub7[i])))
    cv2.waitKey(0)
    cv2.destroyAllWindows()


In [18]:
#11x11
sub11=[]
for i in range(len(img_list)):
    sub11.append(padded_list11[i]-blur11[i])
    cv2.imshow("11x11Sub Original-Subtracted",np.hstack((padded_list11[i],sub11[i])))
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [22]:
#3 shift images 2 pixels up
shiftUp=[]
for i in range(len(img_list)):
    OperationFunc(5,i,True,False,False,False)
    shiftUp.append(npadded_list5[i])
    cv2.imshow("shiftUp",shiftUp[i])
    cv2.waitKey(0)
    cv2.destroyAllWindows()
   

In [23]:
#3 shift images 3 pixels right
shiftRight=[]
for i in range(len(img_list)):
    OperationFunc(7,i,False,True,False,False)
    shiftRight.append(npadded_list7[i])
    cv2.imshow("shiftRight",shiftRight[i])
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [24]:
#4 Sharp image using mask method. You should sum up the original image and mask version.
sharp=[]
for i in range(len(img_list)):
    OperationFunc(3,i,False,False,True,False)
    sharp.append(npadded_list3[i])
    cv2.imshow("sharp Original-Sharp",np.hstack((padded_list3[i],sharp[i])))
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [26]:
#5 Remove salt and pepper noise by using median filter for salt and pepper image

OperationFunc(3,1,False,False,False,True)
cv2.imshow("SaltAndPepper Original-Filtred",np.hstack((padded_list3[1],npadded_list3[1])))
cv2.waitKey(0)
cv2.destroyAllWindows()


13